In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ir-dataset/data-scrap.pickle


In [ ]:
#data preprocessing will be going on here, bwlo mentioned are the key points:

#taget of removal finding and removing similar symptoms
#stop words removal
#tokenisation
#lemmatization

In [ ]:
import csv
import pickle
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from itertools import combinations
from time import time
from nltk.tokenize import RegexpTokenizer
from collections import OrderedDict

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

with open('/kaggle/input/ir-dataset/data-scrap.pickle', 'rb') as handle:
    dis_symp = pickle.load(handle)

t0=time()
total_symptoms = set() # Stores all unique symptoms
diseases_symptoms_cleaned = OrderedDict() # Key: disease, Value:[List of symptoms]

In [ ]:
size_of_dict = len(dis_symp)
print(size_of_dict)
#for key, value in selected_dict.items():
 #   print(f"{key}: {value}")

545


In [ ]:
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer

# Set the NLTK_DATA environment variable
os.environ["NLTK_DATA"] = "/path/to/nltk_data"

# Download the WordNet data
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Iterate over all disease and preprocess symptoms string and break it into individual symptom
for key in sorted(dis_symp.keys()):
    value = dis_symp[key]
    list_sym = re.sub(r"\[\S+\]", "", value).lower().split(',')
    #list_sym = [item for item in list_sym if item != '']
    temp_sym = list_sym
    list_sym = []
    for sym in temp_sym:
        if len(sym.strip())>0:
            list_sym.append(sym.strip())
    if "none" in list_sym:
        list_sym.remove("none");
    if len(list_sym)==0:
        continue
    temp = list()
    for sym in list_sym:
        sym=sym.replace('-',' ')
        sym=sym.replace("'",'')
        sym=sym.replace('(','')
        sym=sym.replace(')','')
        doc = nlp(sym)
        processed_sym = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_digit])
        total_symptoms.add(sym)
        temp.append(sym)
    diseases_symptoms_cleaned[key] = temp



In [ ]:
total_symptoms = list(total_symptoms)
total_symptoms.sort()
total_symptoms=['label_dis']+total_symptoms

In [ ]:
print(len(diseases_symptoms_cleaned))

543


In [ ]:
# Initialize two dataframes, one for normal dataset and one for combination dataset
df_comb = pd.DataFrame(columns=total_symptoms)
df_norm = pd.DataFrame(columns=total_symptoms)

In [ ]:
import pandas as pd
from itertools import combinations

# Assuming df_norm and df_comb are initialized DataFrame objects

# Read each disease and symptom list, convert into dictionary and add to dataframe
for key, values in diseases_symptoms_cleaned.items():
    key = str.encode(key).decode('utf-8')

    # Populate row for normal
    row_norm = dict({x: 0 for x in total_symptoms})
    for sym in values:
        row_norm[sym] = 1
    row_norm['label_dis'] = key
    df_norm = pd.concat([df_norm, pd.DataFrame([row_norm])], ignore_index=True)

    # Populate rows for combination dataset
    for comb in range(1, len(values) + 1):
        for subset in combinations(values, comb):
            row_comb = dict({x: 0 for x in total_symptoms})
            for sym in list(subset):
                row_comb[sym] = 1
            row_comb['label_dis'] = key
            df_comb = pd.concat([df_comb, pd.DataFrame([row_comb])], ignore_index=True)


In [ ]:
df_comb.to_csv("Combined_Symptom.csv",index=None)
df_norm.to_csv("Individual_symptom.csv",index=None)
